---
title: "React Three Fiber with Theater.js"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["Three.js", "R3F", "Theater.js", "Visualization", "JavaScript", "TypeScript"]
format:
  html:
    code-fold: false
toc-depth: 4
jupyter: python3
highlight-style: github
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
---

# Foreword

React Three Fiber is a React wrapper for Three.js, which allows us to use Three.js in a React application. It provides a way to create and manage 3D scenes using React components, making it easier to integrate 3D graphics into React applications. In this post, we will explore how to use React Three Fiber with [Theater.js](https://www.theatrejs.com/), a library for creating animations and interactive experiences in the browser. Theater.js basically provides a rich CG editor (like Blender) in the browser which lets us translate/scale or otherwise transform 3D objects, use animation curves, and keyframes, etc. 

## Overview of Theater.js

Theater.js is composed of two main parts and many optional extensions.

1. Core: The core of Theater.js is a JavaScript library that provides the runtime for Theater.js code (which is just a wrapper on top of Three.js, which is, by extension, a wrapper of WebGL).

    ```bash
    npm install @theatre/core
    ```
    In fact, when we use Theater to update the position of a 3D object, we explicitly set up a listener in Theater.js that picks up the changes and passes them down to Three.js.

2. Studio: The Studio is a web-based editor that allows users to create and edit Theater.js projects visually. It provides a user interface for manipulating 3D objects, creating animations, and managing scenes. We take this library out in the production version of our app (hence we may only install it as a dev dependency).
    ```bash
    npm install @theatre/studio
    ```
3. Extensions: There are many extensions that let us customize our workflow! For example, [@theater/r3f](https://www.npmjs.com/package/@theatre/r3f) is a UI tool called a "snapshot editor" that extends the functionality of the studio library. It includes the translation tools, rotation tools, and scaling tools that bringing the experience much closer to that of using a 3D editor (like Blender). It works with React Three Fiber created scenes...

Speaking of React Three Fiber, we need to understand what it is and how it differs from Three.js.

## Overview of React Three Fiber

React Three Fiber is a very thin wrapper over Three.js that allows us to use Three.js in a React application. It provides a way to create and manage 3D scenes using React components. 

This means we create our objects (meshes)a\ and other scene elements in a reactive way using React's component system and lifecycle management. There is no need to manually delete meshes (garbage collect) like there is in Three.js because React Three Fiber handles this for us through React's lifecycle management. Let's see Three.js and React Three Fiber side by side. 

**Three.js**
---
<details><summary>Click to expand</summary>

```javascript
import {
  ACESFilmicToneMapping,
  AmbientLight,
  BoxGeometry,
  Mesh,
  MeshStandardMaterial,
  PerspectiveCamera,
  PointLight,
  Scene,
  sRGBEncoding,
  WebGLRenderer,
} from "three";

class Cube extends Mesh {
  constructor() {
    super();

    const geometry = new BoxGeometry();
    const material = new MeshStandardMaterial();
    material.color.set("blue");

    this.geometry = geometry;
    this.material = material;
  }

  update() {
    this.rotation.x += 0.01;
    this.rotation.y += 0.01;
  }

  dispose() {
    this.geometry.dispose();
  }
}

// Create "Canvas"
const scene = new Scene();
const camera = new PerspectiveCamera(
  75,
  window.innerWidth / window.innerHeight,
  0.1,
  1000
);

camera.position.z = 5;

// Add elements
const ambientLight = new AmbientLight();
scene.add(ambientLight);

const pointLight = new PointLight();

pointLight.position.set(10, 10, 10);
scene.add(pointLight);

const cube = new Cube();
scene.add(cube);

// Render
const renderer = new WebGLRenderer({ alpha: true, antialias: true });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.toneMapping = ACESFilmicToneMapping;
renderer.outputEncoding = sRGBEncoding;
renderer.setSize(window.innerWidth, window.innerHeight);

const container = document.getElementById("__next");

if (container) {
  container.appendChild(renderer.domElement);
}

function animate() {
  requestAnimationFrame(animate);
  renderer.render(scene, camera);

  cube.update();
}

animate();
```
</details>

**R3F**
---
<details><summary>Click to expand</summary>

```javascript
import { PerspectiveCamera } from "@react-three/drei";
import { Canvas, useFrame } from "@react-three/fiber";
import { useRef } from "react";
import { Mesh } from "three";

function Cube() {
  const meshRef = useRef<Mesh>(null);

  useFrame(() => {
    if (!meshRef.current) {
      return;
    }

    meshRef.current.rotation.x += 0.01;
    meshRef.current.rotation.y += 0.01;
  });

  return (
    <mesh ref={meshRef}>
      <PerspectiveCamera />
      <boxGeometry />
      <meshStandardMaterial color="blue" />
    </mesh>
  );
}

export default function R3fDemo() {
  return (
    <Canvas>
      <ambientLight />
      <pointLight position={[10, 10, 10]} />
      <Cube />
    </Canvas>
  );
}
```

R3F follows this convention.

* Every child component is appended to it aparent per the Three.js `append` method.
* The only exceptions are the geomeries and materials, which are passed as props to the `<mesh>` component (although in R3F they appear to be nested w.r.t the mesh).
* The `<Canvas>` component is not the HTML Canvas, it's an R3F wrapper component that sets up the Three.js scene, camera, and renderer. It is the root component of a Three.js scene in R3F.
* R3F follows React's same naming convention. React uses lower case for JSX elements that are primitives (like `<div>`, `<h1>`, etc.). R3F follows the same convention, it lowercases the components that are primitive with respect to Three.js (rather than HTML itself). So, `<mesh>` is a React component that represents a Three.js `three.Mesh` object. And, in general, every Three.js object, say `ThreeJsObject`, has a corresponding R3F component like `threeJsObject`.

</details>
